In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sqlalchemy

engine = sqlalchemy.create_engine('sqlite:///stock_etf.db')
data = pd.read_sql_query("SELECT * FROM stockData", engine, parse_dates="Date")
index_data = pd.read_sql_query("SELECT * FROM ETF", engine, parse_dates="Date")


In [3]:
def beta(data: pd.DataFrame, ticker: str) -> float:

    spy_ticker1 = pd.concat(
        [index_data.SPY[-500::], data[-500::]], axis=1)
    spy_ticker1.columns = ["SPY", ticker]
    long_data_pct = np.log(spy_ticker1/spy_ticker1.shift())
    ticker1_cov = long_data_pct.cov().iloc[0, 1]
    ticker1_var = long_data_pct["SPY"].var()
    stock_beta = ticker1_cov/ticker1_var

    return stock_beta

In [4]:
def EMA(data: pd.DataFrame, period: int) -> pd.DataFrame:
    exp_moving_avg = data.ewm(span=period, adjust=False).mean()
    return exp_moving_avg

In [5]:
def ratios(pairs: str):
    pair_list = pairs.split()
    ratios = pd.DataFrame()
    for pair in pair_list:
        stock = pair.split('/')
        ratio = pd.DataFrame({
            f'{stock[0]}/{stock[1]}': data[f'{stock[0]}'] / data[f'{stock[1]}']
        })
        ratios = pd.concat([ratios, ratio], axis=1)
    return ratios

In [7]:
def ratio_beta(pairs: str):
    pair_list = pairs.split()
    betas = pd.DataFrame()
    
    for pair in pair_list:
        stock = pair.split('/')
        long_beta = beta(data[f'{stock[0]}'], f'{stock[0]}').round(2)
        short_beta = beta(data[f'{stock[1]}'], f'{stock[1]}').round(2)
        
        beta_value = long_beta / short_beta
        
        beta_ratio = pd.DataFrame({
            f'{stock[0]}/{stock[1]}': beta_value.round(2)
        }, index=[0])
        betas = pd.concat([betas, beta_ratio], axis=1)
    return betas

In [9]:
def pct_returns(given_df: pd.DataFrame):
    c_c_returns = ((given_df - given_df.shift(1)) / given_df.shift(1))*100
    return c_c_returns

In [11]:
def watchlist(pairs: str, c_c_returns: pd.DataFrame, ratios: pd.DataFrame, ratio_beta: pd.DataFrame):
    pair_list = pairs.split()
    descriptive_statistics = pd.DataFrame()
    
    for pair in pair_list:
        stock = pair.split('/')
        
        pair_statistics = pd.DataFrame({
            pair: {
                'Long': stock[0],
                'Short': stock[1],
                'Beta': ratio_beta[pair].iloc[-1].round(2),
                'Ratio': ratios[pair].iloc[-1].round(2),
                'EMA20': EMA(ratios[pair], 20).iloc[-1].round(2),
                'EMA60': EMA(ratios[pair], 60).iloc[-1].round(2),
                'EMA200': EMA(ratios[pair], 200).iloc[-1].round(2),
                'Mean': c_c_returns[pair].mean().round(3),
                'STD': c_c_returns[pair].std().round(2),
                'Z_Score': ((c_c_returns[pair].iloc[-1] - c_c_returns[pair].mean()) / c_c_returns[pair].std()).round(2),
                'Kurtosis': c_c_returns[pair].kurtosis().round(2),
                'Skewness': c_c_returns[pair].skew().round(2),
                'Min': c_c_returns[pair].min().round(2),
                'Max': c_c_returns[pair].max().round(2)
                
            }
        })
        
        descriptive_statistics = pd.concat([descriptive_statistics, pair_statistics], axis=1)
    
    return descriptive_statistics.T

Tables: Dividend_watchlist ETF No_div_watchlist pair_statistics stockData

In [74]:
results = pd.read_sql_query("""SELECT * FROM pair_statistics WHERE Beta > 1 AND Min > -30 AND Max < 30 AND STD < 7 AND Volatility < 5""", engine)

In [75]:
results

,Long,Short,Beta,Ratio,EMA20,EMA60,EMA200,Volatility,Correlation,Mean,STD,Z_Score,Kurtosis,Skewness,Min,Max
0,A,ACGL,1.78,1.56,1.58,1.59,1.64,0.48,0.85,0.009,1.86,-0.82,3.39,0.16,-11.40,11.48
1,A,AKAM,1.27,1.36,1.34,1.27,1.24,0.10,0.80,0.056,2.03,-0.73,19.99,1.15,-16.37,21.62
2,A,AN,1.01,0.91,0.92,0.92,0.91,0.19,0.60,0.046,2.44,-1.42,6.81,0.49,-17.50,19.07
3,A,ARW,1.02,1.13,1.15,1.15,1.09,0.06,0.87,0.039,1.74,-0.15,9.94,0.56,-12.41,15.21
4,A,BJ,1.66,1.83,1.90,1.92,1.88,0.28,0.79,0.010,2.67,-0.44,8.17,0.09,-18.78,18.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10208,ZTS,NBIX,1.98,1.11,1.21,1.31,1.47,0.17,0.73,0.014,2.70,-2.59,5.52,-0.05,-19.35,17.35
10209,ZTS,POST,2.11,1.50,1.62,1.76,1.92,0.37,0.86,0.020,1.99,-3.78,5.74,-0.30,-15.21,10.62
10210,ZTS,TTWO,1.01,1.02,1.12,1.17,1.22,0.07,0.89,0.009,2.23,-2.37,7.86,0.21,-16.29,16.23
10211,ZTS,VRTX,1.52,0.38,0.41,0.43,0.47,0.03,0.86,0.029,2.16,-3.25,14.71,0.25,-17.49,25.57


In [76]:
def pair_plot(results :pd.DataFrame):

    for index, row in results.iterrows():
        
        buy_ticker = row['Long']
        sell_ticker = row['Short']
       
        pair_data = pd.read_sql_query(
                    f"SELECT Date, {buy_ticker}, {sell_ticker} FROM stockData", engine, parse_dates="Date")
        
        data = pd.DataFrame(
                            {
                                'Date': pair_data["Date"],
                                'ratio': (pair_data[buy_ticker] / pair_data[sell_ticker]),
                                'spread': (pair_data[buy_ticker] - pair_data[sell_ticker]).round(2)
                            }
                        )
    
        fig, axes = plt.subplots(
        3, figsize=(15, 15), sharex=True)
        axes[0].set_title(f"{buy_ticker} // {sell_ticker}")
        sns.lineplot(data=pair_data, x="Date", y=buy_ticker, ax=axes[0], label=buy_ticker)
        sns.lineplot(data=pair_data, x="Date", y=sell_ticker, ax=axes[0], label=sell_ticker)
        axes[1].set_title(f"Ratio")
        sns.lineplot(data=data, x="Date", y="ratio", ax=axes[1])
        axes[2].set_title(f"Spread")
        sns.lineplot(data=data, x="Date", y="spread", ax=axes[2])
        plt.tight_layout(pad=1)

        fig.savefig(r'd://StockMarket/Beta_over_1/'+f"{buy_ticker}__{sell_ticker}")
        plt.close()

In [77]:
pair_plot(results)

MemoryError: In RendererAgg: Out of memory

Error in callback <function _draw_all_if_interactive at 0x0000016CE6CA20C0> (for post_execute), with arguments args (),kwargs {}:


MemoryError: In RendererAgg: Out of memory

MemoryError: In RendererAgg: Out of memory

<Figure size 1500x1500 with 3 Axes>